In [139]:
import numpy as np

In [140]:
k_b = 1.38e-16 # Kelvin
a = 7.56e-15 # erg cm^-3
ion = 1.26
electron = 1.15
K = 1e13 # erg cm^-3
m_H = 1.66053907e-24

In [141]:
N_rho = int(input('Enter desired number of points for density: '))
N_T = N_rho #Note: N_rho must be equal to N_T, as such the desired number of points for density will automatically equal the desired number of points for temperature
rho = np.logspace(np.log10(1), np.log10(1e4), num = N_rho)
T = np.logspace(np.log10(1e6), np.log10(1e8), num = N_T)

In [142]:
def pressure(rho, T):
    return ((k_b * rho * T)/(ion * m_H)) + (1/3) * a * T**4 + K*(rho)**(5/3) * electron**(-5/3)

In [147]:
print(f'This is density: {rho}')
print(f'This is temperature: {T}')
print(f'This is pressure: {pressure(rho,T)}')

This is density: [1.00000000e+00 1.20679264e+00 1.45634848e+00 1.75751062e+00
 2.12095089e+00 2.55954792e+00 3.08884360e+00 3.72759372e+00
 4.49843267e+00 5.42867544e+00 6.55128557e+00 7.90604321e+00
 9.54095476e+00 1.15139540e+01 1.38949549e+01 1.67683294e+01
 2.02358965e+01 2.44205309e+01 2.94705170e+01 3.55648031e+01
 4.29193426e+01 5.17947468e+01 6.25055193e+01 7.54312006e+01
 9.10298178e+01 1.09854114e+02 1.32571137e+02 1.59985872e+02
 1.93069773e+02 2.32995181e+02 2.81176870e+02 3.39322177e+02
 4.09491506e+02 4.94171336e+02 5.96362332e+02 7.19685673e+02
 8.68511374e+02 1.04811313e+03 1.26485522e+03 1.52641797e+03
 1.84206997e+03 2.22299648e+03 2.68269580e+03 3.23745754e+03
 3.90693994e+03 4.71486636e+03 5.68986603e+03 6.86648845e+03
 8.28642773e+03 1.00000000e+04]
This is temperature: [1.00000000e+06 1.09854114e+06 1.20679264e+06 1.32571137e+06
 1.45634848e+06 1.59985872e+06 1.75751062e+06 1.93069773e+06
 2.12095089e+06 2.32995181e+06 2.55954792e+06 2.81176870e+06
 3.08884360e+06

In [144]:
rho_0 = float(input('Enter desired density value in g cm^-3: '))
T_0 = float(input('Enter desired temperature value in K: '))

rho_i = rho[np.searchsorted(rho,rho_0) - 1]
rho_ip1 = rho[np.searchsorted(rho,rho_0)]
T_j = T[np.searchsorted(T,T_0) - 1]
T_jp1 = T[np.searchsorted(T,T_0)]
print(f'({rho_0}, {T_0})')
print(f'({rho_i}, {T_j})')
print(f'({rho_ip1}, {T_j})')
print(f'({rho_i}, {T_jp1})')
print(f'({rho_ip1}, {T_jp1})')

(7000.0, 49000000.0)
(6866.488450042998, 47148663.6345739)
(8286.427728546843, 47148663.6345739)
(6866.488450042998, 51794746.79231212)
(8286.427728546843, 51794746.79231212)


You can rewrite the solution to a bilinear interpolation problem as a multilinear polynomial:
$$P(\rho, T) \approx a \rho T + b T + c \rho + d$$
where the coefficients are found by solving the linear system:
$$\begin{bmatrix}
1 & \rho_1 & T_1 & \rho_1 T_1 \\
1 & \rho_1 & T_2 & \rho_1 T_2 \\
1 & \rho_2 & T_1 & \rho_2 T_1 \\
1 & \rho_2 & T_2 & \rho_2 T_2  
\end{bmatrix}
\begin{bmatrix} 
d \\
c \\ 
b \\
a
\end{bmatrix} =
\begin{bmatrix}
P(\rho_1, T_1) \\
P(\rho_1, T_2) \\
P(\rho_2, T_1) \\
P(\rho_2, T_2) \\
\end{bmatrix}
$$

In [145]:
L = np.array([
    [1, rho_i, T_j, rho_i*T_j],
    [1, rho_i, T_jp1, rho_i*T_jp1],
    [1, rho_ip1, T_j, rho_ip1*T_j],
    [1, rho_ip1, T_jp1, rho_ip1*T_jp1],
])
m = np.array([pressure(rho_i, T_j), pressure(rho_i, T_jp1), pressure(rho_ip1, T_j), pressure(rho_ip1, T_jp1)])

x = np.linalg.solve(L,m)
a1 = x[3]
b = x[2]
c = x[1]
d = x[0]
print(f'a = {a1}, b = {b}, c = {c}, d= {d}')

a = 65956779.64012529, b = 1223172529.6704123, c = 5091608928335067.0, d= -1.5355234709799027e+19


In [146]:
def bilinear_interpolation(a1,b,c,d, rho_0, T_0):
    return a1*(rho_0)*(T_0) + b*(T_0) + c*(rho_0) + d
print(bilinear_interpolation(a1,b,c,d,rho_0,T_0))
print(f'{pressure(rho_i,T_j)} < {bilinear_interpolation(a1,b,c,d,rho_0,T_0)} < {pressure(rho_ip1,T_jp1)}')
print(pressure(rho_0,T_0))

4.296913865906327e+19
4.1017137513506456e+19 < 4.296913865906327e+19 < 5.520758483085274e+19
4.293011447294101e+19
